<a href="https://colab.research.google.com/github/JonatanSiracusa/download-historical-series/blob/main/download_hist_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Historical prices download


In this notebook we will be downloading the historical series of a list of stocks.

1. Byma´s prices downloaded from Yahoo Finance. 


In order to get the desired results, the next steps must be followed:

1. Open the Excel file named 'tickers.xlsx' located in the same folder of this program: 
	* Complete the `'ticker_byma'` column.
	* Complete the `'ticker_yahoo'` column. 
2. Set the `'start_date'` variable in the section 1 of this program.
3. Set the `'NOMBRE_OUTPUT'` variable in the section 1 of this program. Data series will be saved and named by the value set in this variable.


The next steps will be followed in order to implement the ***Project***:

1. Kick-off: Libraries Importing, Variables Setup and Functions.

2. Data Loading

3. Data Cleaning

4. Data Transformation

5. Results saving


***************************



# 1. Kick-off: Libraries Importing, Variables Setup and Functions

In [ ]:
import numpy as np
import pandas as pd
import scipy
import math
import random
import time
import datetime as dt
from datetime import datetime

import yfinance as yf

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings

In [ ]:
# medimos el tiempo de ejecucion del programa
star_time = time.time()

start_date = dt.datetime(1994, 1, 1)
end_date = dt.datetime.now()

RUEDAS_ANIO = 252
NOMBRE_OUTPUT_1 = 'historical-Adj_prices-byma'
NOMBRE_OUTPUT_2 = 'historical-Adj_prices_plus-byma'
EXPORTAR_DATOS = False

warnings.simplefilter("ignore")

In [3]:
def ticker_simple_return():
	return list(map(lambda elem: elem + '_sr', tickers))

def ticker_log_return(): 
	return list(map(lambda elem: elem + '_lr', tickers))

def ticker_volat(): 
	return list(map(lambda elem: elem + '_v40', tickers))


In [4]:
def convert_to_dataframe(data):
    """
    Esta función toma una entrada `data` y verifica si es una Serie o un DF.
    Si es una Serie de Pandas, la convierte en un DF y renombre la columna.
    Si es un DF, lo devuelve sin cambios.
    """	
    if isinstance(data, pd.Series):
        # Convertir la Serie a DataFrame
        data = pd.DataFrame(data)
        data.columns.values[0] = tickers[0]
        return data
    elif isinstance(data, pd.DataFrame):
        # Si ya es un DataFrame, no se hace nada
        return data
    else:
        raise ValueError("La entrada no es una pandas.Series ni un pandas.DataFrame.")


def get_volatility(ticker, df):
	"""
	Esta funcion busca en el DF y devuelve la volatilidad de las ultimas 40 ruedas anualizada del ticker ingresado.
	"""
	indice = tickers.index(ticker)
	variable = ticker_volat()[indice]
	valor = df.loc[:, variable].iloc[-1]
	return valor


# 2. Data Loading

In [5]:
# Completamos las listas Tickers y Tickers_name
xlsx = pd.ExcelFile('./tickers.xlsx')
df1 = pd.read_excel(xlsx, 'Hoja1')

tickers = df1.iloc[:, 0].tolist()
tickers_yahoo = df1.iloc[:, 1].tolist()

df1.info()
print(tickers)
print(tickers_yahoo)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ticker_byma   22 non-null     object
 1   ticker_yahoo  22 non-null     object
dtypes: object(2)
memory usage: 484.0+ bytes
['Index', 'ALUA', 'BBAR', 'BMA', 'BYMA', 'CEPU', 'COME', 'CRES', 'CVH', 'EDN', 'GGAL', 'LOMA', 'MIRG', 'PAMP', 'SUPV', 'TECO2', 'TGNO4', 'TGSU2', 'TRAN', 'TXAR', 'VALO', 'YPFD']
['^MERV', 'ALUA.BA', 'BBAR.BA', 'BMA.BA', 'BYMA.BA', 'CEPU.BA', 'COME.BA', 'CRES.BA', 'CVH.BA', 'EDN.BA', 'GGAL.BA', 'LOMA.BA', 'MIRG.BA', 'PAMP.BA', 'SUPV.BA', 'TECO2.BA', 'TGNO4.BA', 'TGSU2.BA', 'TRAN.BA', 'TXAR.BA', 'VALO.BA', 'YPFD.BA']


In [6]:
# Descargamos las cotizaciones de todos los tickers
data = round(yf.download(tickers_yahoo, start=start_date, end=end_date)['Adj Close'], 2)

data.info()

[*********************100%***********************]  22 of 22 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6997 entries, 1996-10-08 00:00:00+00:00 to 2024-11-01 00:00:00+00:00
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ALUA.BA   5525 non-null   float64
 1   BBAR.BA   6190 non-null   float64
 2   BMA.BA    6190 non-null   float64
 3   BYMA.BA   1794 non-null   float64
 4   CEPU.BA   6193 non-null   float64
 5   COME.BA   6190 non-null   float64
 6   CRES.BA   6190 non-null   float64
 7   CVH.BA    1748 non-null   float64
 8   EDN.BA    4289 non-null   float64
 9   GGAL.BA   6043 non-null   float64
 10  LOMA.BA   1705 non-null   float64
 11  MIRG.BA   6190 non-null   float64
 12  PAMP.BA   5143 non-null   float64
 13  SUPV.BA   2065 non-null   float64
 14  TECO2.BA  6190 non-null   float64
 15  TGNO4.BA  4384 non-null   float64
 16  TGSU2.BA  6190 non-null   float64
 17  TRAN.BA   6038 non-null   float64
 18  TXAR.BA   6190 non-null   float64
 19  VALO.BA   4156 non-nu

# 3. Data Cleaning

In [7]:
# La descarga trae un indice de fecha. Seteamos el formato de la fecha del indice
data.index = pd.to_datetime(data.index).strftime('%Y-%m-%d')

data = convert_to_dataframe(data)

# Cambiamos los nombres de los tickers que acepta la fuente de datos por los que deseamos
for ticker_y, name in zip(tickers_yahoo, tickers):
	data.rename(columns={ticker_y: name}, inplace=True)

# Agregamos un indice "n" en la columna 1
data.insert(0, 'n', 1, allow_duplicates=False)
data['n'] = data['n'].cumsum()

# Guardo la columna del ticker 1 y la inserto en pos=1
col_1 = data.pop(tickers[0])
data.insert(1, tickers[0], col_1)

# Reemplazamos los missing values y negativos por 0
data.fillna(0, inplace=True)
data[data < 0] = 0

prices_v2 = data.copy()
prices_v2.info()
prices_v2

<class 'pandas.core.frame.DataFrame'>
Index: 6997 entries, 1996-10-08 to 2024-11-01
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   n       6997 non-null   int64  
 1   Index   6997 non-null   float64
 2   ALUA    6997 non-null   float64
 3   BBAR    6997 non-null   float64
 4   BMA     6997 non-null   float64
 5   BYMA    6997 non-null   float64
 6   CEPU    6997 non-null   float64
 7   COME    6997 non-null   float64
 8   CRES    6997 non-null   float64
 9   CVH     6997 non-null   float64
 10  EDN     6997 non-null   float64
 11  GGAL    6997 non-null   float64
 12  LOMA    6997 non-null   float64
 13  MIRG    6997 non-null   float64
 14  PAMP    6997 non-null   float64
 15  SUPV    6997 non-null   float64
 16  TECO2   6997 non-null   float64
 17  TGNO4   6997 non-null   float64
 18  TGSU2   6997 non-null   float64
 19  TRAN    6997 non-null   float64
 20  TXAR    6997 non-null   float64
 21  VALO    6997 non-null   flo

Ticker,n,Index,ALUA,BBAR,BMA,BYMA,CEPU,COME,CRES,CVH,...,MIRG,PAMP,SUPV,TECO2,TGNO4,TGSU2,TRAN,TXAR,VALO,YPFD
Date,,,,,,,,,,,,,,,,,,,,,
1996-10-08,1,590.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-10-09,2,583.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-10-10,3,585.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-10-11,4,584.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996-10-14,5,584.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28,6993,1851914.0,826.0,5320.0,9140.0,310.0,1290.0,241.25,1090.0,4700.0,...,22300.0,3120.0,2060.0,2020.0,3010.0,5110.0,1860.0,776.0,332.0,29600.0
2024-10-29,6994,1845639.0,832.0,5310.0,9140.0,309.0,1300.0,240.50,1125.0,4790.0,...,22700.0,3080.0,2095.0,2000.0,3285.0,5190.0,2010.0,786.0,333.5,28700.0
2024-10-30,6995,1837535.0,826.0,5220.0,9050.0,304.0,1305.0,242.00,1130.0,4855.0,...,23150.0,3105.0,2055.0,2085.0,3290.0,5190.0,2050.0,789.0,331.0,28450.0


# 4. Data Transformation

A cada activo le agregamos los siguientes calculos: 
* Rendimiento simple (o discreto), respecto de rueda anterior.
* Rendimiento logaritmico (o continuo), respecto de rueda anterior.
* Volatilidad (o desvio estandar) de las ultimas 40 ruedas, anualizado.

In [8]:
# Comenzamos a transformar los datos con una nueva version del DF
prices_v3 = prices_v2.copy()

In [9]:
# Insertamos los calculos de rendimientos y volatildiad
for i in range(len(tickers)):
	
	# Seleccionamos el ticker
	asset = tickers[i]

	# Buscamos la posicion de la columna del ticker
	pos = prices_v3.columns.get_loc(asset)

	# Asignamos el nombre a la nueva col
	col_sr = ticker_simple_return()[i]
	col_lr = ticker_log_return()[i]
	col_v40 = ticker_volat()[i]

	# Insertamos una columna luego del ticker, con el nombre correspondiente al calculo y el ticker
	prices_v3.insert(pos+1, col_sr, np.nan)
	prices_v3.insert(pos+2, col_lr, np.nan)
	prices_v3.insert(pos+3, col_v40, np.nan)

	# Calculo los Simple Return diarios
	prices_v3[col_sr] = (prices_v3[asset] / prices_v3[asset].shift(1)) - 1

	# Calculo los Log Return diarios
	prices_v3[col_lr] = np.log(prices_v3[asset] / prices_v3[asset].shift(1))

	# Calculo el Desvio St de las ultimas 40 ruedas anualizado
	prices_v3[col_v40] = (prices_v3[col_lr].rolling(window=40).std()) * (RUEDAS_ANIO ** (1/2))

prices_v3.fillna(0, inplace=True)

print(get_volatility('GGAL', prices_v3))
prices_v3

0.3486790112150608


Ticker,n,Index,Index_sr,Index_lr,Index_v40,ALUA,ALUA_sr,ALUA_lr,ALUA_v40,BBAR,...,TXAR_lr,TXAR_v40,VALO,VALO_sr,VALO_lr,VALO_v40,YPFD,YPFD_sr,YPFD_lr,YPFD_v40
Date,,,,,,,,,,,,,,,,,,,,,
1996-10-08,1,590.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1996-10-09,2,583.0,-0.011864,-0.011935,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1996-10-10,3,585.0,0.003431,0.003425,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1996-10-11,4,584.0,-0.001709,-0.001711,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1996-10-14,5,584.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-28,6993,1851914.0,-0.011144,-0.011207,0.273568,826.0,0.011016,0.010956,0.279871,5320.0,...,-0.017880,0.311983,332.0,0.001508,0.001507,0.225398,29600.0,-0.011686,-0.011755,0.349865
2024-10-29,6994,1845639.0,-0.003388,-0.003394,0.269149,832.0,0.007264,0.007238,0.280222,5310.0,...,0.012804,0.313617,333.5,0.004518,0.004508,0.224529,28700.0,-0.030405,-0.030877,0.343947
2024-10-30,6995,1837535.0,-0.004391,-0.004401,0.267646,826.0,-0.007212,-0.007238,0.277318,5220.0,...,0.003810,0.302848,331.0,-0.007496,-0.007524,0.218619,28450.0,-0.008711,-0.008749,0.327964


# 5. Results Saving

Guardamos en formato .csv y .xlsx la serie de datos solo con cotizaciones y la serie de datos con cotizaciones y calculos de rendimientos y volatilidad.

In [10]:
if EXPORTAR_DATOS:
	# Guardamos en un .csv
	prices_v2.to_csv('./' + NOMBRE_OUTPUT_1 + '.csv')

	# Guardamos en un .xlsx
	with pd.ExcelWriter('./' + NOMBRE_OUTPUT_1 + '.xlsx') as writer:
		prices_v2.to_excel(writer, sheet_name='Sheet1', index=True)


	# Guardamos en un .csv
	prices_v3.to_csv('./' + NOMBRE_OUTPUT_2 + '.csv')

	# Guardamos en un .xlsx
	with pd.ExcelWriter('./' + NOMBRE_OUTPUT_2 + '.xlsx') as writer:
		prices_v3.to_excel(writer, sheet_name='Sheet1', index=True)


		ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
		print(f'Data has already been exported. Date: {ahora}.')

else:
	ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
	print(f'No data has been exported. Date: {ahora}.')


end_time = time.time()
execution_time = end_time - star_time
print(f'\nExecution time: {round(execution_time, 2)} seconds.')

Data has already been exported. Date: 2024-11-03 00:12:32.

Execution time: 16.33 seconds.
